In [ ]:
import pandas as pd

In [ ]:
prot_counts_file = "data/Proteomics Counts.csv"     
rna_file = "data/RNA Seq Stats.csv"            
prot_stats_file = "data/Proteomics Stats.csv"      

prot_counts = pd.read_csv(prot_counts_file)   
rna_df = pd.read_csv(rna_file)                
prot_stats = pd.read_csv(prot_stats_file)     

### STEP1: Find the UniProt for RNA seq and the Proteomics file

In [ ]:
import pandas as pd
import mygene

mg = mygene.MyGeneInfo()

# Clean the ENSG（.1, .2…）
ensg_ids = rna_df["Unnamed: 0"].str.replace(r"\.\d+$", "", regex=True).tolist()

# Use ENSG to find UniProt + GeneSymbol
rna_map = mg.querymany(
    ensg_ids,
    scopes="ensembl.gene",
    fields="symbol,uniprot.Swiss-Prot",
    species="human"
)

rna_map_df = pd.DataFrame(rna_map)

rna_map_df["UniProt"] = rna_map_df["uniprot"].apply(
    lambda x: x.get("Swiss-Prot") if isinstance(x, dict) else x
    if isinstance(x, str) else None
)

rna_map_df = rna_map_df[["query","symbol","UniProt"]].rename(
    columns={"query":"ENSG","symbol":"GeneSymbol"}
)

# Merge the table
rna_df["ENSG_clean"] = rna_df["Unnamed: 0"].str.replace(r"\.\d+$", "", regex=True)
rna_df = rna_df.merge(rna_map_df, left_on="ENSG_clean", right_on="ENSG", how="left")

rna_df = rna_df.drop(columns=["ENSG","ENSG_clean"])

rna_df.to_csv("data/RNA_Seq_with_Symbol.csv", index=False)


In [ ]:
# Use proteomics_counts to find the Uniprot
prot_map = prot_counts[["PG.ProteinGroups", "PG.ProteinNames"]].rename(
    columns={"PG.ProteinGroups":"UniProt", "PG.ProteinNames":"EntryName"}
).drop_duplicates()

# Prot Stats + Prot Counts merge by Entryname
prot_stats = prot_stats.rename(columns={"G.ProteinNames":"EntryName"})
prot_stats_merged = prot_stats.merge(prot_map, on="EntryName", how="left")

out_file = "data/Proteomics_Stats_with_UniProt.csv"
prot_stats_merged.to_csv(out_file, index=False)

### STEP2: Merged RNA seq and Proteomics File using the UniProt

In [ ]:
rna_file = pd.read_csv("data/RNA_Seq_with_Symbol.csv")
prot_file = pd.read_csv("data/Proteomics_Stats_with_UniProt.csv")

# Merged two files based on the Uniprot
merged = prot_df.merge(rna_df, on="UniProt", how="inner")

out_file = "data/RNA_Proteomics_Merged.csv"
merged.to_csv(out_file, index=False)

### STEP3: Filter the merged file with log2fc>0 and p/q value <0

In [ ]:
merged_file = "data/RNA_Proteomics_Merged.csv"
merged = pd.read_csv(merged_file)

# Apply the filters
filtered = merged[
    (merged["log2FoldChange"] > 0) &
    (merged["Log2FC"] > 0) &
    (merged["Welch's T-test q-value ZFC3H1 KO"] < 0.05) &
    (merged["padj"] < 0.05)
]

out_file = "data/RNA_Proteomics_Filtered.csv"
filtered.to_csv(out_file, index=False)


#### Find NM_id and use that to find CDS

In [ ]:
import pandas as pd

rna_df = pd.read_csv("data/RNA_Proteomics_Filtered.csv")
rna_df = rna_df.rename(columns={"Unnamed: 0": "ENSG"})

mane_df = pd.read_csv("MANE.GRCh38.v1.4.summary.txt", sep="\t", comment = None)

mane_select = mane_df[mane_df["MANE_status"] == "MANE Select"]

ensg_to_nm = dict(zip(mane_select["symbol"], mane_select["RefSeq_nuc"]))

rna_df["NM_ID"] = rna_df["GeneSymbol"].map(ensg_to_nm)

rna_df.to_csv("data/RNA_Proteomics_Filtered.csv", index=False)


In [ ]:
from Bio import Entrez, SeqIO

Entrez.email = "sijin@ad.unc.edu"

def fetch_cds(nm_id):
    """Fetch CDS nucleotide sequence for a given NM accession"""
    try:
        handle = Entrez.efetch(db="nucleotide", id=nm_id, rettype="gb", retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()

        # Find CDS feature
        for feature in record.features:
            if feature.type == "CDS":
                seq = feature.extract(record.seq)
                return str(seq)
        return None
    except Exception as e:
        print(f"Failed for {nm_id}: {e}")
        return None

# Fetch CDS for each NM_ID
cds_dict = {}
for nm in rna_df["NM_ID"].dropna():
    cds_dict[nm] = fetch_cds(nm)

# Add column back
rna_df["CDS"] = rna_df["NM_ID"].map(cds_dict)

# Save
rna_df.to_csv("data/RNA_Proteomics_Filtered.csv", index=False)


### STEP4: RNA targets not affected at protein level (log2FC>0 RNA-seq and padj<0.05 RNA-seq)

In [ ]:
# apply filters
filtered = merged[
    (merged["log2FoldChange"] > 0) &
    (merged["padj"] < 0.05)
]

out_file = "data/RNA_Proteomics_RNAlevel.csv"
filtered.to_csv(out_file, index=False)

#### Fine CDS of the RNA Level file using NM_ID

In [ ]:
import pandas as pd

rna_seq_df = pd.read_csv("data/RNA_Proteomics_RNAlevel.csv")
rna_seq_df = rna_seq_df.rename(columns={"Unnamed: 0": "ENSG"})

# Load MANE summary file-download from NCBI website
mane_df = pd.read_csv("MANE.GRCh38.v1.4.summary.txt", sep="\t", comment = None)

# Keep only MANE Select rows
mane_select = mane_df[mane_df["MANE_status"] == "MANE Select"]

# Build GeneSymbol→ NM mapping
ensg_to_nm = dict(zip(mane_select["symbol"], mane_select["RefSeq_nuc"]))

# Map NM_ID using your 'GeneSymbol' column
rna_seq_df["NM_ID"] = rna_seq_df["GeneSymbol"].map(ensg_to_nm)

# Save back to the same file
rna_seq_df.to_csv("data/RNA_Proteomics_RNAlevel.csv", index=False)

In [ ]:
from Bio import Entrez, SeqIO

Entrez.email = "sijin@ad.unc.edu"

rna_seq_df["CDS"] = rna_seq_df["NM_ID"].map(cds_dict)

# Save
rna_seq_df.to_csv("data/RNA_Proteomics_RNAlevel.csv", index=False)

#### Find the CAI value for each gene

### STEP5: RNA targets affected at protein level (log2FC>0 RNA and protein and padj<0.05)

#### 5.1 Find NM_id; 5.2 Find CDS use NM_id

In [ ]:
# apply filters
filtered = merged[
    (merged["Log2FC"] > 0) &
    (merged["Welch's T-test q-value ZFC3H1 KO"] < 0.05)
]

out_file = "data/RNA_Proteomics_Proteinlevel.csv"
filtered.to_csv(out_file, index=False)

In [ ]:
prot_seq_df = pd.read_csv("data/RNA_Proteomics_Proteinlevel.csv")
prot_seq_df = prot_seq_df.rename(columns={"Unnamed: 0": "ENSG"})

mane_df = pd.read_csv("MANE.GRCh38.v1.4.summary.txt", sep="\t", comment = None)

mane_select = mane_df[mane_df["MANE_status"] == "MANE Select"]

ensg_to_nm = dict(zip(mane_select["symbol"], mane_select["RefSeq_nuc"]))

prot_seq_df["NM_ID"] = prot_seq_df["GeneSymbol"].map(ensg_to_nm)

prot_seq_df.to_csv("data/RNA_Proteomics_Proteinlevel.csv", index=False)

In [ ]:
# Use nm_id to find CDS
prot_seq_df["CDS"] = prot_seq_df["NM_ID"].map(cds_dict)
prot_seq_df.to_csv("data/RNA_Proteomics_Proteinlevel.csv", index=False)

### STEP6: All protein's distribution

In [ ]:
import pandas as pd
import mygene

# Input files
prot_counts_file = "data/Proteomics Counts.csv"
prot_stats_file  = "data/Proteomics Stats.csv"
prot_counts = pd.read_csv(prot_counts_file)
prot_stats  = pd.read_csv(prot_stats_file)

# Map the Uniprot
prot_map = prot_counts[["PG.ProteinGroups", "PG.ProteinNames"]].rename(
    columns={"PG.ProteinGroups":"UniProt", "PG.ProteinNames":"EntryName"}
).drop_duplicates()

# Merge the file
prot_stats = prot_stats.rename(columns={"G.ProteinNames":"EntryName"})
prot_stats_merged = prot_stats.merge(prot_map, on="EntryName", how="left")

# Find the Genesymbol by uniprot 
mg = mygene.MyGeneInfo()
res = mg.querymany(
    prot_stats_merged["UniProt"].dropna().tolist(),
    scopes="uniprot",
    fields="symbol",
    species="human"
)

map_df = pd.DataFrame(res)[["query","symbol"]].rename(
    columns={"query":"UniProt","symbol":"GeneSymbol"}
)

prot_stats_final = prot_stats_merged.merge(map_df, on="UniProt", how="left")

out_file = "data/All_Prot_Level.csv"
prot_stats_final.to_csv(out_file, index=False)


#### Find nm_id and use nm_id to find CDS

In [ ]:
all_df = pd.read_csv("data/All_Prot_Level.csv")
mane_df = pd.read_csv("MANE.GRCh38.v1.4.summary.txt", sep="\t", comment = None)
mane_select = mane_df[mane_df["MANE_status"] == "MANE Select"]
ensg_to_nm = dict(zip(mane_select["symbol"], mane_select["RefSeq_nuc"]))

all_df["NM_ID"] = all_df["GeneSymbol"].map(ensg_to_nm)

all_df.to_csv("data/All_Prot_Level.csv", index=False)

In [ ]:
all_df["NM_ID"].dropna().drop_duplicates().to_csv("data/my_ids.txt", index=False, header=False)

#then upload the nm_id's to NCBI and download the CDS in sequence.txt